In [1]:
import numpy as np 
import os
import pandas as pd
from scipy import ndarray
import skimage as sk
from skimage import io
import random
from skimage import transform
from skimage import img_as_ubyte
import shutil

In [2]:
input_dir = "/home/mimus/apiais/data/images/ANPR/training/original_plate_loc"
target_dir = "/home/mimus/apiais/data/images/ANPR/training/masks_plate_loc"


input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".jpg") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)


Number of samples: 15788
/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_0.jpg | /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_0.jpg
/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_0_0.jpg | /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_0_0.jpg
/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_0_0_augmented_image_9516.jpg | /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_0_0_augmented_image_9516.jpg
/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_0_10.jpg | /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_0_10.jpg
/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_0_10_augmented_image_14992.jpg | /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_0_10_augmented_image_14992.jpg
/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_0_11.jpg | /home/mimus/api

In [3]:
def split_train_test(data,test_ratio):
    shuffled_indices=np.random.permutation(len(data))
    test_set_size=int(len(data)*test_ratio)
    test_indices=shuffled_indices[:test_set_size]
    train_indices=shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [4]:
df = pd.DataFrame(input_img_paths) 
train_set,test_set=split_train_test(df,0)
print(len(train_set),"train +",len(test_set),"test")

15788 train + 0 test


In [5]:
#here we need to copy the test files to another folder for later use

test_files = test_set.values.tolist()
test_file_names = []
test_file_names2 = []
for i in test_files:
    test_file_names.append(i[0])
    test_file_names2.append(i[0].replace('/original_', '/masks_'))
#print(test_file_names[0:10])
#print(test_file_names2[0:10])
for name in test_file_names2:
    shutil.move(name, name.replace('/training/', '/test/'))
for name in test_file_names:
    shutil.move(name, name.replace('/training/', '/test/'))

In [6]:
train_files = train_set.values.tolist()
train_file_names = []
train_file_names2 = []
for i in train_files:
    train_file_names.append(i[0])
    train_file_names2.append(i[0].replace('/original_', '/masks_'))

In [10]:
train_file_names

['/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_83_28.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_328_29.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_0_10.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_846_32.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_465_0.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_81_33_augmented_image_8926.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_522_32_augmented_image_10071.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_89_40.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_60_36_augmented_image_14102.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_loc/original_51_41.jpg',
 '/home/mimus/apiais/data/images/ANPR/training/original_plate_l

In [8]:
def random_rotation(image_array: ndarray, image_array2: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-10, 10)
    return sk.transform.rotate(image_array, random_degree), sk.transform.rotate(image_array2, random_degree)

def random_noise(image_array: ndarray, image_array2: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array), image_array2

# dictionary of the transformations we defined earlier
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise
}

original_img = train_file_names
mask_img = train_file_names2
num_files_desired = 20000

final_val = len(original_img)

# find all files paths from the folder
#print("control 1")
num_generated_files = len(original_img)
counter = 0
while num_generated_files <= num_files_desired:

    # random image from the folder
    image_path = original_img[counter]
    image_path2 = mask_img[counter]
    # read image as an two dimensional array of pixels
    image_to_transform = sk.io.imread(image_path)
    image_to_transform2 = sk.io.imread(image_path2)
    # random num of transformation to apply
    num_transformations_to_apply = random.randint(1, len(available_transformations))
    num_transformations = 0
    transformed_image = None
    while num_transformations <= num_transformations_to_apply:
       # print(num_transformations,num_transformations_to_apply)
        # random transformation to apply for a single image
        key = random.choice(list(available_transformations))
        transformed_image, transformed_image2 = available_transformations[key](image_to_transform, image_to_transform2)
        #aqui deberiamso poner la misma lista de archivos pero con mask
        num_transformations+= 1
    new_file_path = '%s_augmented_image_%s.jpg' % (image_path[:-4], num_generated_files)    
    new_file_path2 = '%s_augmented_image_%s.jpg' % (image_path2[:-4], num_generated_files)    

    # write image to the disk

    io.imsave(new_file_path, img_as_ubyte(transformed_image))
    io.imsave(new_file_path2, img_as_ubyte(transformed_image2))
    num_generated_files += 1
    counter += 1
    print(counter, num_generated_files)


1 15789
2 15790
3 15791


/home/mimus/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_328_29_augmented_image_15789.jpg is a low contrast image
/home/mimus/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_846_32_augmented_image_15791.jpg is a low contrast image


4 15792
5 15793


/home/mimus/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_465_0_augmented_image_15792.jpg is a low contrast image


6 15794
7 15795
8 15796


...

4205 19993
4206 19994
4207 19995
4208 19996
4209 19997


/home/mimus/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_242_28_augmented_image_12218_augmented_image_19996.jpg is a low contrast image
/home/mimus/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_305_32_augmented_image_19997.jpg is a low contrast image


4210 19998
4211 19999


/home/mimus/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: /home/mimus/apiais/data/images/ANPR/training/masks_plate_loc/masks_230_28_augmented_image_13454_augmented_image_19998.jpg is a low contrast image


4212 20000
4213 20001
